In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from config import api_key
from citipy import citipy

In [2]:
cities = []

# Creating a set of random lat and lng combinations
lats = np.random.uniform(low=-90, high=90, size=1500)
lngs = np.random.uniform(low=-180, high=180, size=1500)
lat_lng = zip(lats, lngs)

# Finding nearest city for each lat-lng combination
for value in lat_lng:
    city = citipy.nearest_city(value[0], value[1]).city_name 

    # If the city is unique, then adding it to the cities list
    if city not in cities: 
        cities.append(city)
        
# Calculating total city count                              
total_cities = len(cities)
total_cities

628

In [4]:
city_id = []
city_name_list = []
country_list = []
cloudiness_list = []
date_list = []
humidity_list = []
lat_list = []
lng_list = []
max_temp_list = []
wind_speed_list = []
i = 1

print("-----------------------------")
print("Beginning Data Retrieval")
print("-----------------------------")

base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{base_url}appid={api_key}&units={units}&q="

for city in cities:
    try:
        response = requests.get(query_url + city).json()
        ID = response["id"] 
        city_id.append(response["id"])
        city_name_list.append(response["name"])
        country_list.append(response["sys"]["country"])
        cloudiness_list.append(response["clouds"]["all"])
        date_list.append(response["dt"])
        humidity_list.append(response["main"]["humidity"])
        lat_list.append(response["coord"]["lat"])
        lng_list.append(response["coord"]["lon"])
        max_temp_list.append(response['main']['temp_max'])
        wind_speed_list.append(response["wind"]["speed"])
        print(f"Processing {i} of {total_cities}: {city} with City ID:{ID}")
        i+=1
        
    except(KeyError, IndexError):
        print("City not found. Skipping...")
        
print("-----------------------------") 
print("Ending Data Retrieval")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval
-----------------------------
Processing 1 of 628: vaini with City ID:4032243
Processing 2 of 628: rikitea with City ID:4030556
Processing 3 of 628: carnarvon with City ID:2074865
Processing 4 of 628: saint-augustin with City ID:6137462
Processing 5 of 628: mataura with City ID:6201424
Processing 6 of 628: ibicui with City ID:3461602
Processing 7 of 628: ribeira grande with City ID:3372707
Processing 8 of 628: swakopmund with City ID:3352844
Processing 9 of 628: bilma with City ID:2446796
Processing 10 of 628: yellowknife with City ID:6185377
Processing 11 of 628: port alfred with City ID:964432
Processing 12 of 628: albany with City ID:5106841
Processing 13 of 628: atuona with City ID:4020109
Processing 14 of 628: jamestown with City ID:5122534
Processing 15 of 628: hobart with City ID:2163355
City not found. Skipping...
Processing 16 of 628: busselton with City ID:2075265
Processing 17 of 628: nikolskoye with City ID:546105
Proce